##Ductile Micro Grinding - Extended
#Inputs

Dc is tool diameter

Ae is radial stepover

Cc is the grain concentration according to FEPA. C145 for ordinary metal bond, C75 for ordinary resin bond from Haefeli

dg is the average grain size in microns, i.e. D33 = 33 microns

E is the Modulus of Elasticity, H is the hardness and K_c is the toughness

Vc is tool surface speed

In [4]:
import math
import ipywidgets as widgets


slider_rpm = widgets.IntSlider(value=100000, min=10, max=250000, description='Spindle rpm')
slider_diameter = widgets.FloatSlider(value=3, min=0.1, max=10, description='Dc mm')

slider_chip_aspect_ratio = widgets.IntSlider(value=10, min=2, max=50, description='r (5-15)')
slider_grain_density = widgets.FloatLogSlider(value=500, base=10, min=1, max=6, step=0.001, description='C grain density')

slider_E = widgets.IntSlider(value=168, min=10, max=1000, step=1, description='E GPa')
slider_H = widgets.FloatSlider(value=11, min=1, max=80, step=0.1, description='H GPa')
slider_Kc = widgets.FloatSlider(value=0.7, min=0.1, max=20, step=0.1, description='Kc MPa.m^1/2')

slider_crit_chip = widgets.FloatLogSlider(disabled=True, value=0.00001, base=10, min=-6, max=0.1, step = 0.01, description='h_cu-max')

slider_stepover = widgets.FloatLogSlider(value=0.001, base=10, min=-6, max=1, step = 0.01,description='Ae mm')

output_crit_chip_bifano = widgets.FloatText(disabled=True, description='Bifano h_cu-max')
output_crit_chip_huang = widgets.FloatText(disabled=True, description='Huang h_cu-max')
output_surface_speed = widgets.FloatText(disabled=True, description='Vc m/min')
output_feedrate_side = widgets.FloatText(disabled=True, description='Side Vf mm/s')

def compute_surface_speed(*ignore):
    output_surface_speed.value = round(math.pi*slider_diameter.value * slider_rpm.value/1000)

def compute_feedrates_crit_chip(*ignore):
    output_feedrate_side.value = round((output_surface_speed.value*slider_crit_chip.value**2)/((4/(slider_grain_density.value*slider_chip_aspect_ratio.value))*math.sqrt(slider_stepover.value/slider_diameter.value)),3)

def compute_crit_chip(*ignore):

    E = slider_E.value*10e9
    H = slider_H.value*10e9
    Kc = slider_Kc.value*10e6

    output_crit_chip_bifano.value = 0.15*(E/H)*((Kc/H)**2)
    output_crit_chip_huang.value = 8.7*(math.sqrt(H/E))*((Kc/H)**2)

slider_rpm.observe(compute_surface_speed, 'value')
slider_diameter.observe(compute_surface_speed, 'value')

slider_E.observe(compute_crit_chip, 'value')
slider_H.observe(compute_crit_chip, 'value')
slider_Kc.observe(compute_crit_chip, 'value')

slider_rpm.observe(compute_feedrates_crit_chip, 'value')
slider_diameter.observe(compute_feedrates_crit_chip, 'value')
slider_chip_aspect_ratio.observe(compute_feedrates_crit_chip, 'value')
slider_grain_density.observe(compute_feedrates_crit_chip, 'value')
slider_crit_chip.observe(compute_feedrates_crit_chip, 'value')
slider_stepover.observe(compute_feedrates_crit_chip, 'value')


display(slider_rpm)
display(slider_diameter)
display(slider_chip_aspect_ratio)
display(slider_grain_density)
display(slider_E)
display(slider_H)
display(slider_Kc)
display(slider_crit_chip)
display(slider_stepover)


IntSlider(value=100000, description='Spindle rpm', max=250000, min=10)

FloatSlider(value=3.0, description='Dc mm', max=10.0, min=0.1)

IntSlider(value=10, description='r (5-15)', max=50, min=2)

FloatLogSlider(value=500.0, description='C grain density', max=6.0, min=1.0, step=0.001)

IntSlider(value=168, description='E GPa', max=1000, min=10)

FloatSlider(value=11.0, description='H GPa', max=80.0, min=1.0)

FloatSlider(value=0.7, description='Kc MPa.m^1/2', max=20.0, min=0.1)

FloatLogSlider(value=1e-05, description='h_cu-max', disabled=True, max=0.1, min=-6.0, step=0.01)

FloatLogSlider(value=0.001, description='Ae mm', max=1.0, min=-6.0, step=0.01)

#Results

In [5]:
display(output_surface_speed)
display(output_feedrate_side)


FloatText(value=1132.0, description='Vc m/min', disabled=True)

FloatText(value=0.008, description='Side Vf mm/s', disabled=True)